<a href="https://colab.research.google.com/github/tomonari-masada/course-nlp2020/blob/master/08_document_classification_with_RNN(example).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 08 RNNを使った文書分類
* RNNの出力を文書の潜在表現として利用し、文書分類を行う

## 08-01 torchtextを使ってIMDbデータを読み込む
* ここでIMDbデータセットの読み込みにつかう`torchtext.datasets`については、下記を参照。
 * https://torchtext.readthedocs.io/en/latest/datasets.html

### 実験の再現性確保のための設定など
* https://pytorch.org/docs/stable/notes/randomness.html

In [1]:
import random
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext import datasets
from torchtext.data import Field, LabelField, BucketIterator

SEED = 123

random.seed(SEED)
torch.manual_seed(SEED)
torch.set_deterministic(True)

### torchtextのフィールド
* TEXTフィールドと、LABELフィールドという２種類のFieldオブジェクトのインスタンスを作る。
 * Fieldクラスの詳細については[ここ](https://github.com/pytorch/text/blob/master/torchtext/data/field.py)を参照。
* TEXTフィールドは、テキストの前処理の仕方を決めておくのに使う。
* LABELフィールドは、ラベルの前処理に使う。

In [2]:
TEXT = Field(tokenize="spacy")
LABEL = LabelField()

### IMDbデータセットをダウンロードした後、前処理しつつ読み込む
* ダウンロードはすぐ終わるが、解凍に少し時間がかかる。
* また、TEXTフィールドでspaCyのtokenizationを使うように設定したので、少し時間がかかる。

In [3]:
train_valid_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

### テストセット以外の部分を訓練データと検証データに分ける

In [21]:
train_data, valid_data = train_valid_data.split(split_ratio=0.8)
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 20000
Number of validation examples: 5000
Number of testing examples: 25000


### データセットの語彙とラベルを作る
* TEXTラベルのほうでは、最大語彙サイズを指定する。

In [5]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [6]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [7]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 231681), (',', 220513), ('.', 189315), ('and', 125380), ('a', 124937), ('of', 115088), ('to', 107165), ('is', 87305), ('in', 70009), ('I', 61800), ('it', 61302), ('that', 56289), ('"', 50495), ("'s", 49496), ('this', 48345), ('-', 42165), ('/><br', 40918), ('was', 39731), ('as', 34712), ('with', 34376)]


### デバイスの取得

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### ミニバッチを取り出すためのiteratorを作る

In [9]:
BATCH_SIZE = 100

train_iterator = BucketIterator(train_data, batch_size=BATCH_SIZE, device=device,
                                     sort_within_batch=True, shuffle=True, sort_key=lambda x: len(x.text))
valid_iterator = BucketIterator(valid_data, batch_size=BATCH_SIZE, device=device)
test_iterator = BucketIterator(test_data, batch_size=BATCH_SIZE, device=device)

### 定数の設定
* 単語埋め込みの次元もRNNの隠れ状態の次元も128に増やす。

In [10]:
INPUT_DIM = len(TEXT.vocab)
NUM_CLASS = len(LABEL.vocab)
EMBED_DIM = 128
HIDDEN_DIM = 128
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

### モデルの定義
* GRUを使う。
* レイヤー数は2にする。
* gradientのクリッピングはしないことにしたので削除。

In [11]:
class RNNTextSentiment(nn.Module):
  def __init__(self, emb_dim, hid_dim,
               num_class, vocab_size, padding_idx, p=0.0):
    super().__init__()

    self.input_dim = vocab_size
    self.emb_dim = emb_dim
    self.hid_dim = hid_dim
    self.padding_idx = padding_idx

    self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=padding_idx)
    self.rnn = nn.GRU(emb_dim, hid_dim, num_layers=2)
    self.fc = nn.Linear(hid_dim * 2, num_class)
    self.dropout = nn.Dropout(p=p)

  def forward(self, src):
    # srcの形は[単語列長, バッチサイズ]

    embedded = self.dropout(self.embedding(src))
    # embeddedの形は[単語列長, バッチサイズ, 埋め込み次元数]

    outputs, hidden = self.rnn(embedded)
    # outputsの形は[単語列長, バッチサイズ, 隠れ状態の次元数]
    # hiddenの形は[レイヤー数, バッチサイズ, 隠れ状態の次元数]

    # 平均を正確に計算する
    mask = (src != self.padding_idx)
    mean_outputs = (outputs * mask.unsqueeze(2)).sum(0) / mask.sum(0).unsqueeze(1)
    hidden = hidden[-1,:,:].squeeze()
    # mean_outputsの形は[バッチサイズ, 隠れ状態の次元数]
    # hiddenの形は[バッチサイズ, 隠れ状態の次元数]
    output = self.fc(torch.cat((mean_outputs, hidden), dim=1))

    return output

* モデルのインスタンスを得る

In [12]:
model = RNNTextSentiment(EMBED_DIM, HIDDEN_DIM, NUM_CLASS, INPUT_DIM,
                         padding_idx=PAD_IDX, p=0.5).to(device)

* 重みの初期化もデフォルトのままで使う。

### 最適化アルゴリズムの設定

In [13]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

パラメータの数を数えてみる。

In [14]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,398,914 trainable parameters


### 文書分類の損失関数の設定

In [15]:
criterion = nn.CrossEntropyLoss()

### 訓練用の関数
* gradientクリッピングはしないことにする

In [16]:
def train(model, iterator, optimizer, criterion):
  model.train()
  epoch_loss = 0.
  epoch_acc = 0.
  for batch in iterator:

    optimizer.zero_grad()
    output = model(batch.text)
    loss = criterion(output, batch.label)
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()
    epoch_acc += (output.argmax(1) == batch.label).sum().item()

  return epoch_loss / len(iterator), epoch_acc / len(iterator.dataset)

### 評価用の関数

In [17]:
def evaluate(model, iterator, criterion):
  model.eval()
  epoch_loss = 0.
  epoch_acc = 0.
  with torch.no_grad():
    for batch in iterator:
      output = model(batch.text)
      loss = criterion(output, batch.label)
      epoch_loss += loss.item()
      epoch_acc += (output.argmax(1) == batch.label).sum().item()

  return epoch_loss / len(iterator), epoch_acc / len(iterator.dataset)

In [18]:
def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time / 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
  return elapsed_mins, elapsed_secs

### 学習の実行

In [19]:
N_EPOCHS = 20

for epoch in range(1, N_EPOCHS + 1):

  start_time = time.time()
  train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
  valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
  end_time = time.time()
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  print(f'Epoch {epoch} | time in {epoch_mins} minutes, {epoch_secs} seconds')
  print(f'\tLoss {train_loss:.4f} (train)\t|\tAcc {train_acc * 100:.1f}% (train)')
  print(f'\tLoss {valid_loss:.4f} (valid)\t|\tAcc {valid_acc * 100:.1f}% (valid)')

Epoch 1 | time in 0 minutes, 21 seconds
	Loss 0.5992 (train)	|	Acc 66.3% (train)
	Loss 0.5069 (valid)	|	Acc 76.1% (valid)
Epoch 2 | time in 0 minutes, 21 seconds
	Loss 0.4455 (train)	|	Acc 79.4% (train)
	Loss 0.4076 (valid)	|	Acc 82.7% (valid)
Epoch 3 | time in 0 minutes, 21 seconds
	Loss 0.3592 (train)	|	Acc 84.4% (train)
	Loss 0.4468 (valid)	|	Acc 80.7% (valid)
Epoch 4 | time in 0 minutes, 21 seconds
	Loss 0.3084 (train)	|	Acc 86.8% (train)
	Loss 0.3131 (valid)	|	Acc 88.1% (valid)
Epoch 5 | time in 0 minutes, 21 seconds
	Loss 0.2734 (train)	|	Acc 88.9% (train)
	Loss 0.2964 (valid)	|	Acc 88.6% (valid)
Epoch 6 | time in 0 minutes, 21 seconds
	Loss 0.2549 (train)	|	Acc 89.7% (train)
	Loss 0.3079 (valid)	|	Acc 88.8% (valid)
Epoch 7 | time in 0 minutes, 21 seconds
	Loss 0.2258 (train)	|	Acc 90.9% (train)
	Loss 0.2982 (valid)	|	Acc 89.1% (valid)
Epoch 8 | time in 0 minutes, 21 seconds
	Loss 0.2086 (train)	|	Acc 91.9% (train)
	Loss 0.3062 (valid)	|	Acc 88.6% (valid)
Epoch 9 | time in 0 minu

### テストデータで評価

In [20]:
print('Checking the results of test dataset...')
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'\tLoss: {test_loss:.5f}(test)\t|\tAcc: {test_acc * 100:.2f}%(test)')

Checking the results of test dataset...
	Loss: 0.42087(test)	|	Acc: 88.10%(test)
